## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# Recognition accuracy experiments on the ORL face recognition dataset with a regularization parameter of $1.0$

##### Importing general modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import sklearn
import scipy.stats as st
import scipy.io as sio
import time
import matplotlib.image as mpimg
import matplotlib as mpl

##### Importing modules wtitten by the author

In [2]:
from LinearDiscriminantAnalysis import LinearDiscriminantAnalysis
from LDATrainingDataPreparation import TrainingData
from NearestNeighbor import NearestNeighbor

##### Setup

In [3]:
C  = 40
spc = 10 # sample per class
K = 4
maxVqIteration = 100
width = 92
height = 112
reg = 1

##### Load dataset

In [4]:
path = '../datasets/orl_faces'
X = np.zeros ([C, spc, height, width])

for c in range (C):
    for i in range (spc):
        file = path + '/s' + str(c+1) + '/' + str(i+1) + '.pgm' 
        X[c,i,:,:] = mpimg.imread (file) / 255
        img = np.zeros ([height,width, 3])
        img[:,:,0] = img[:,:,1] = img[:,:,2] = X[c,i]
        #plt.imshow (img)
        #plt.show()

##### Defining a function for computing accuracy on a specific dataset (i.e. a fold)

In [5]:
def compute_accuracy(XTrain, yTrain, XTest, yTest, solver, S1, S2, regularization, min_svd):
    np.random.seed(1)
    td = TrainingData(XTrain, yTrain)

    clusAlg = sklearn.cluster.KMeans()
    clusAlg.max_iter = maxVqIteration
    td.findSubclasses(np.ones(C, dtype = int)*K, clusAlg)
    
    #for i in range(td.C):
    #    for j in range (td.Ki[i]):
    #        X = td.subclassMeans[i,j,:]
    #        img = np.zeros ([height,width, 3])
    #        img[:,:,0] = img[:,:,1] = img[:,:,2] = np.reshape(X, [height, width])
    #        plt.imshow (img)
    #        plt.show()    
    
    lda = LinearDiscriminantAnalysis(td, solver, S1, S2, regularization, min_svd)
    lda.fitFeatureExtractor()
    obj = lda.objective()
    
    XTrainFtr = lda.transform (XTrain)
    XTestFtr  = lda.transform (XTest)
    accNN = NearestNeighbor (XTrainFtr, yTrain, XTestFtr, yTest)
    return accNN, obj

##### Defining a function for computing average accuracy on 10 folds

In [6]:
def compute_average_accuracy (solver, S1, S2, reg):
    nFolds = 10
    accNN = np.zeros ([nFolds])
    obj = np.zeros ([nFolds])
    np.random.seed(1)
    for fold in range (nFolds):
        XTrain1 = X[:,0:fold,:,:]
        XTrain2 = X[:,fold+1:,:,:]
        XTrain  = np.concatenate((XTrain1, XTrain2), axis = 1)
        XTest   = X[:,fold,:,:]
        yTrain   = np.repeat (range (C), spc-1)
        yTest   = range (C)
        XTrain = XTrain.reshape ([-1, height * width])
        XTest = XTest.reshape ([-1, height * width]) 
        accNN[fold], obj[fold] = compute_accuracy(XTrain, yTrain, XTest, yTest, 
                                                  solver, S1, S2, reg, 0.0001)
        print ('At fold ', fold, ' accNN=', accNN[fold], 'objective=', obj[fold])

    return accNN, obj

In [9]:
solver = 'svd'
S1 ='Sb'
S2 = 'St'
accNN, obj = compute_average_accuracy (solver, S1, S2, reg)
for (name, acc) in [('accNN', accNN)]:
    print ('')
    acc *= 100
    print (name , 'Mean: ', np.mean (acc), 'Std of Mean Accuracy: ', 
           np.std (acc), 'Objective:', np.mean(obj), 'Std of Objective:', np.std(obj))

At fold  0  accNN= 0.975 objective= 44.04858448842951
At fold  1  accNN= 1.0 objective= 43.881405635342674
At fold  2  accNN= 1.0 objective= 43.87552294692139
At fold  3  accNN= 0.975 objective= 43.71763290320771
At fold  4  accNN= 1.0 objective= 43.58877835772613
At fold  5  accNN= 1.0 objective= 43.942108058934785
At fold  6  accNN= 1.0 objective= 44.121398153423044
At fold  7  accNN= 0.975 objective= 43.762383995250204
At fold  8  accNN= 0.975 objective= 44.1483491170807
At fold  9  accNN= 0.925 objective= 44.17297225590961

accNN Mean:  98.25 Std of Mean Accuracy:  2.25 Objective: 43.92591359122257 Std of Objective: 0.18770809397005267


In [8]:
solver = 'ghiasi_svd'
S1 ='Sb'
S2 = 'St'
accNN, obj = compute_average_accuracy (solver, S1, S2, reg)
for (name, acc) in [('accNN', accNN)]:
    print ('')
    acc *= 100
    print (name , 'Mean: ', np.mean (acc), 'Std of Mean Accuracy: ', 
           np.std (acc), 'Objective:', np.mean(obj), 'Std of Objective:', np.std(obj))

At fold  0  accNN= 0.975 objective= 44.04858448842953
At fold  1  accNN= 1.0 objective= 43.88140563534269
At fold  2  accNN= 1.0 objective= 43.8755229469214
At fold  3  accNN= 0.95 objective= 43.71763290320773
At fold  4  accNN= 0.975 objective= 43.58877835772611
At fold  5  accNN= 1.0 objective= 43.942108058934814
At fold  6  accNN= 0.95 objective= 44.12139815342312
At fold  7  accNN= 0.975 objective= 43.762383995250175
At fold  8  accNN= 0.95 objective= 44.1483491170807
At fold  9  accNN= 0.95 objective= 44.172972255909535

accNN Mean:  97.25 Std of Mean Accuracy:  2.0766559657295187 Objective: 43.92591359122259 Std of Objective: 0.1877080939700543


### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>